# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
import pickle
sys.path.append('../../src')
import DISData as DD
import pickle

## SQL 연결

In [5]:
doUT = DD.SQL()

host:  ········
user:  ········
password:  ········


SQL connection done


## query 작성

In [18]:
query1 = (f'''CREATE TABLE SYNTHESEYES_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(500) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(500) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        Gaze_3D VARCHAR(200) NOT NULL,
        Head_3D VARCHAR(200) NOT NULL);''')

query2 = (f'''CREATE TABLE SYNTHESEYES_landmark_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(500) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(500) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        pupil_3d VARCHAR(1000) NOT NULL,
        pupil_2d VARCHAR(1000) NOT NULL,
        
        iris_3d VARCHAR(1000) NOT NULL,
        iris_2d VARCHAR(1000) NOT NULL,
        lids_3d VARCHAR(1000) NOT NULL,
        lids_2d VARCHAR(1000) NOT NULL);''')
query3 = (f'''INSERT INTO SYNTHESEYES_info 
        (img_dir, img_format, img_width, img_height, 
        color_space, label_dir, label_format, Gaze_3D, Head_3D) VALUES 
        (%s, %s, %s, %s,  %s, %s, %s, %s, %s)''')
query4 = (f'''INSERT INTO SYNTHESEYES_landmark_info 
        (img_dir, img_format, img_width, img_height, 
        color_space, label_dir, label_format, pupil_3d, pupil_2d, 
        iris_3d, iris_2d, lids_3d, lids_2d) VALUES 
        (%s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s, %s, %s)''')

create table done
create table done


## query 실행

In [6]:
try:
    doUT.create_table(query2)
    doUT.create_table(query1)

except Exception as e:
    print(f"Error during table creation: {e}")

In [7]:
# label 경로
label_path = 'Z:/SYNTHESEYES/data/RGB/labeldata/'
image_path = 'Z:/SYNTHESEYES/data/RGB/rawdata/'
base_folder = os.listdir(label_path)

value_list1 = []
value_list2 = []
error_list = []
for num, folder in enumerate(base_folder):
    label_files = os.listdir(os.path.join(label_path,folder))
    try:
        for label in label_files:
            label_dir = os.path.join(label_path,folder,label)
            img_dir = os.path.join(image_path,folder,label.replace('pkl','png'))
            with open(label_dir, 'rb') as f:
                texts = pickle.load(f)
            pupil_2d = str(texts['ldmks']['ldmks_pupil_2d']).replace('[','').replace(']','').replace('(','').replace(')','')
            lids_2d = str(texts['ldmks']['ldmks_lids_2d']).replace('[','').replace(']','').replace('(','').replace(')','')
            iris_2d = str(texts['ldmks']['ldmks_iris_2d']).replace('[','').replace(']','').replace('(','').replace(')','')
            pupil_3d = str(texts['ldmks']['ldmks_pupil_3d']).replace('[','').replace(']','').replace('(','').replace(')','')
            lids_3d = str(texts['ldmks']['ldmks_lids_3d']).replace('[','').replace(']','').replace('(','').replace(')','')
            iris_3d = str(texts['ldmks']['ldmks_iris_3d']).replace('[','').replace(']','').replace('(','').replace(')','')
            gaze_3D = str(texts['look_vec']).replace('[','').replace(']','').replace('(','').replace(')','')
            head_3D = str(texts['head_pose']).replace('[','').replace(']','').replace('(','').replace(')','')
            label_format = 'pkl'
            label_dir = label_dir.replace('Z:','DataBase').replace('\\','/')
            # print(img_dir)
            image = cv2.imread(img_dir)
            img_width = image.shape[1]
            img_height = image.shape[0]
            img_dir = img_dir.replace('Z:','DataBase').replace('\\','/')
            img_format = 'png'
            color_space = 'RGB'

            value1 = (img_dir, img_format, img_width, img_height, 
            color_space, label_dir, label_format, gaze_3D, head_3D)
            # print(value1)
            value2 = (img_dir, img_format, img_width, img_height, 
            color_space, label_dir, label_format, pupil_3d, pupil_2d, 
            iris_3d, iris_2d, lids_3d, lids_2d)

            # print(value)
            value_list1.append(value1)
            value_list2.append(value2)

            if len(value_list1) >= 500:
                print(value_list1[0])
                doUT.insert_dataset_values(query3, value_list1)
                value_list1 = []
                print(f'{num} / {len(base_folder)}')
            if len(value_list2) >= 500:
                doUT.insert_dataset_values(query4, value_list2)
                value_list2 = []
                print(f'{num} / {len(base_folder)}')
    except:
        error_list.append(label_dir)

if len(value_list1) != 0:
    doUT.insert_dataset_values(query3, value_list1)
    # print(value_list2)
    value_list1 = []
    print('done')      
if len(value_list2) != 0:
    doUT.insert_dataset_values(query4, value_list2)
    # print(value_list2)
    value_list2 = []
    print('done')      
else:
    value_list1 = []
    print('done')
        

('DataBase/SYNTHESEYES/data/RGB/rawdata/f01/f01_1002_0.1963_0.3927.png', 'png', 120, 80, 'RGB', 'DataBase/SYNTHESEYES/data/RGB/labeldata/f01/f01_1002_0.1963_0.3927.pkl', 'pkl', '-0.3991550803184509, -0.14731863141059875, -0.9049708843231201', '0.9683144688606262, -1.6611007680467083e-08, 0.24973425269126892, 0.06689619272947311, 0.9634552597999573, -0.25938189029693604, -0.24060776829719543, 0.26786941289901733, 0.9329276084899902')
insert 500 done
0 / 10
insert 500 done
0 / 10
('DataBase/SYNTHESEYES/data/RGB/rawdata/f01/f01_2554_-0.0000_-0.3927.png', 'png', 120, 80, 'RGB', 'DataBase/SYNTHESEYES/data/RGB/labeldata/f01/f01_2554_-0.0000_-0.3927.pkl', 'pkl', '0.39503559470176697, -0.007880485616624355, -0.9186320304870605', '0.9999082684516907, 2.193307535591771e-09, -0.013498697429895401, 0.0016909426776692271, 0.9921227097511292, 0.12525612115859985, 0.013392369262874126, -0.1252669245004654, 0.9920323491096497')
insert 500 done
0 / 10
insert 500 done
0 / 10
('DataBase/SYNTHESEYES/data/

In [13]:
doUT.connect_end()

SQL connection closed
